In [ ]:
import os
import pickle
import json
import numpy as np

In [ ]:
IO_NET_sinks = ['java.io',  'SQL', 'okhttp3', 'SharedPreferences']

interested_var = {'java.io' : 0,
                 'write': 0,
                 'SQL': -10, # should be params
                 'okhttp3': -10,
                 'SharedPreferences': 1}

loose_compare = [' match',
                ' contains']

strict_compare = [' startsWith',
                 ' endsWith',
                 'indexOf']

text_extract = [' opt',
               ' substring']


def parse_result(path):
    f = open(path)
    l = []
    for line in f.readlines():
        l.append(line)
    f.close()
    ss = ''.join(l).replace('\n', ',')
    ss = '[' + ss[:-1] + ']'
    return json.loads(ss)

def is_interest_sink(mthd):
    for s in IO_NET_sinks:
        if s in mthd:
            return s
        
    return False

def get_tv_pos(unit, tv): # 0,1,2: pos; -1: None
    s = tv.find('[')
    if s >= 0:
        tv_list = tv[tv.find('[') + 1 : tv.rfind(']')]
        tv_list = [s.strip for s in tv_list.split(',')]
        rp = unit.rfind(')')
        lp = unit.find(')>(') + 3
        s = unit[lp : rp]
        param_list = [s.strip() for s in s.split(',')]

        tv_pos = []

        for i in range(len(param_list)):
            if param_list[i] in tv:
                tv_pos.append(i)
        if len(tv_pos) > 0:
            return tv_pos
        return [-1,]
    else:
        rp = unit.rfind(')')
        lp = unit.find(')>(') + 3
        s = unit[lp : rp]
        param_list = [s.strip() for s in s.split(',')]
    #     print(param_list)
        for i in range(len(param_list)):
            if param_list[i] == tv:
                return [i,]
        return [-1,]

def deduplicate(json_list):
    dd_json = []
    dd_json_set = set()
    for js in json_list:
        if str(js) in dd_json_set:
            pass
        else:
            dd_json.append(js)
            dd_json_set.add(str(js))
    
    return dd_json


def is_interest_apk(p):
    json_list = parse_result(p)
    
    for js in json_list:
        low_source = js['source']['unit'].lower()
        if 'org.json.jsonobject' in low_source and 'optstring' in low_source or 'java.util.map' in low_source and 'get(' in low_source:
            continue
        rec_sinks = js['sinks']
        for s in rec_sinks:
            if is_interest_sink(s['unit']):
                return deduplicate(json_list)
        else:
            continue
    return None

def get_mthd_type(s):
    u = s['unit']
    mthd = u[u.find('.<') + 2 : u.rfind(')>(') + 1]
    for lc in loose_compare:
        if lc in mthd:
            return 0
    
    for sc in strict_compare:
        if sc in mthd:
            return 1
    
    for te in text_extract:
        if te in mthd:
            return 2
    
    return -1

def eval_type(m):
    if m[1] and m[2]: return 4 # strict format + text extract
    if m[0] and m[2]: return 3  # loose format + text extract
    if m[1]: return 2  # Strict format only
    if m[0]: return 1  # loose format only
    return 0  # No validate

def get_result(json_list):
    
    group_ori_source = {}

    for r in json_list[::-1]:# record {'sinks':..., 'source': ...}
        osource = r['originsource']['unit']
        group_ori_source.setdefault(osource, list([])).append(r)

    exist_type = np.zeros(5)

    for r in group_ori_source.values():
        r_mthd_type = [0, 0, 0]
        for record in r:
            if 'android.content.ClipboardManager' in record['source']['unit']:
                r_mthd_type = [0, 0, 0]
            for s in record['sinks']:

                u = s['unit']
                mtype = get_mthd_type(s)
                if mtype >= 0:
                    r_mthd_type[mtype] = 1
                    continue

                mthd = is_interest_sink(u)
                if not mthd: continue
                tv = s['taint_var']
                vpos = get_tv_pos(u, tv)
                if interested_var[mthd] in vpos or interested_var[mthd] == -10 and max(vpos) >= 0:
                    exist_type[eval_type(r_mthd_type)] = 1


    idx = 0
    while sum(exist_type) > 1:
        exist_type[idx] = 0
        idx += 1

    if sum(exist_type) == 1:
        return exist_type.argmax()
    else: return None

In [ ]:
apk_path = 'res/' # path of the output result of the jar
apk_list = set([f for dp, dn, filenames in os.walk(apk_path) for f in filenames])

result = np.zeros(5)
for t in apk_list:
    if t == 'appWithCB' or t == 'TimeOuts.txt' or t.startswith('.'): continue
    _ = is_interest_apk(apk_path + t)
    if _:
        d = get_result(_)
        if d is not None:
            result[d] += 1

result        

In [ ]:
apk_path = '' # path of the output result of the jar

_ = is_interest_apk(apk_path)
if _:
    print(get_result(_))